In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

2023-05-05 13:21:49.982599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-05 13:21:49.982665: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Import the data

In [2]:
training = pd.read_csv("training_data.csv")
test = pd.read_csv("testing_data.csv")

training = training[["Peruntukan","Pusat_kota","Visibilitas","Bangunan","Luas"]]
test = test[["Peruntukan","Pusat_kota","Visibilitas","Bangunan","Luas"]]

# Change categorical to number 

In [3]:
condition = [  training.Pusat_kota == "Pusat",
             training.Pusat_kota == "Sedang",
              training.Pusat_kota == "Pinggir",
]

value = [3,2,1]

training.Pusat_kota = np.select(condition,value)
####
####

condition = [  training.Visibilitas == "Strategis",
             training.Visibilitas == "Sedang",
              training.Visibilitas == "Kurang",
]

value = [3,2,1]

training.Visibilitas = np.select(condition,value)
#####
#####

condition = [  training.Bangunan == "Bagus",
             training.Bangunan == "Sedang",
             
]

value = [2,1]

training.Bangunan = np.select(condition,value,0)


condition = [ 
            training.Peruntukan == "Pasar",
             training.Peruntukan == "Kantor",
             training.Peruntukan == "Ruko",
             training.Peruntukan == "Taman",
             training.Peruntukan == "Perumahan",
             training.Peruntukan == "Sawah",
             
]

value = [5,4,3,2,1,0]

training.Peruntukan = np.select(condition,value,0)

In [4]:
condition = [  test.Pusat_kota == "Pusat",
             test.Pusat_kota == "Sedang",
              test.Pusat_kota == "Pinggir",
]

value = [3,2,1]

test.Pusat_kota = np.select(condition,value)
####
####

condition = [  test.Visibilitas == "Strategis",
             test.Visibilitas == "Sedang",
              test.Visibilitas == "Kurang",
]

value = [3,2,1]

test.Visibilitas = np.select(condition,value)
#####
#####

condition = [  test.Bangunan == "Bagus",
             test.Bangunan == "Sedang",
             
]

value = [2,1]

test.Bangunan = np.select(condition,value,0)


condition = [ 
            test.Peruntukan == "Pasar",
             test.Peruntukan == "Kantor",
             test.Peruntukan == "Ruko",
             test.Peruntukan == "Taman",
             test.Peruntukan == "Perumahan",
             test.Peruntukan == "Sawah",
             
]

value = [5,4,3,2,1,0]

test.Peruntukan = np.select(condition,value,0)

In [5]:
training

,Peruntukan,Pusat_kota,Visibilitas,Bangunan,Luas
0,1,2,3,1,87580
1,1,2,3,1,87580
2,1,2,3,1,87580
3,1,2,3,1,87580
4,1,2,3,1,57945
...,...,...,...,...,...
123,2,3,3,0,1624
124,2,3,3,0,17033
125,2,3,3,0,2074
126,2,3,3,0,1700


In [6]:
test

,Peruntukan,Pusat_kota,Visibilitas,Bangunan,Luas
0,4,3,3,2,1192
1,4,3,2,2,997
2,5,3,3,2,14060
3,5,3,3,1,6025
4,0,1,1,0,10390
5,0,1,1,0,7745
6,0,1,1,0,5700
7,1,2,3,1,27024
8,1,1,2,1,8740
9,3,3,3,2,3614


# Dataframe to Dataset

In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=4):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Peruntukan')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [8]:
train_ds = df_to_dataset(training)
val_ds = df_to_dataset(test)

2023-05-05 13:21:52.504570: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-05 13:21:52.504646: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-05 13:21:52.504693: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gusanwa): /proc/driver/nvidia/version does not exist
2023-05-05 13:21:52.505260: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Feature Normalizer

In [9]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [10]:
all_inputs = []
encoded_features = []

# Numerical features.
for header in ["Pusat_kota","Visibilitas","Bangunan","Luas"]:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

# Model build

In [11]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(6)(x)

model = tf.keras.Model(all_inputs, output)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [12]:
# Use `rankdir='LR'` to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [13]:
model.fit(train_ds, epochs=100, validation_data=val_ds)

Epoch 1/100
32/32 [==============================] - 1s 11ms/step - loss: 1.8478 - accuracy: 0.2031 - val_loss: 1.7940 - val_accuracy: 0.1538
Epoch 2/100
32/32 [==============================] - 0s 4ms/step - loss: 1.7614 - accuracy: 0.2891 - val_loss: 1.6216 - val_accuracy: 0.3846
Epoch 3/100
32/32 [==============================] - 0s 4ms/step - loss: 1.6034 - accuracy: 0.2891 - val_loss: 1.4707 - val_accuracy: 0.6154
Epoch 4/100
32/32 [==============================] - 0s 4ms/step - loss: 1.4817 - accuracy: 0.4297 - val_loss: 1.3498 - val_accuracy: 0.6923
Epoch 5/100
32/32 [==============================] - 0s 4ms/step - loss: 1.3761 - accuracy: 0.5078 - val_loss: 1.2526 - val_accuracy: 0.6923
Epoch 6/100
32/32 [==============================] - 0s 4ms/step - loss: 1.3460 - accuracy: 0.5156 - val_loss: 1.1828 - val_accuracy: 0.7692
Epoch 7/100
32/32 [==============================] - 0s 4ms/step - loss: 1.2979 - accuracy: 0.5547 - val_loss: 1.1228 - val_accuracy: 0.7692
Epoch 8/100


Epoch 59/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4333 - accuracy: 0.8828 - val_loss: 0.3412 - val_accuracy: 0.9231
Epoch 60/100
32/32 [==============================] - 0s 4ms/step - loss: 0.4395 - accuracy: 0.8438 - val_loss: 0.3363 - val_accuracy: 0.9231
Epoch 61/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4393 - accuracy: 0.8359 - val_loss: 0.3310 - val_accuracy: 0.9231
Epoch 62/100
32/32 [==============================] - 0s 4ms/step - loss: 0.4503 - accuracy: 0.8672 - val_loss: 0.3256 - val_accuracy: 0.9231
Epoch 63/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3981 - accuracy: 0.8828 - val_loss: 0.3261 - val_accuracy: 0.9231
Epoch 64/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4261 - accuracy: 0.8594 - val_loss: 0.3250 - val_accuracy: 0.9231
Epoch 65/100
32/32 [==============================] - 0s 4ms/step - loss: 0.4242 - accuracy: 0.8750 - val_loss: 0.3158 - val_accuracy: 0.9231
Epoch 

In [14]:
model.predict(val_ds)

array([[ -3.3690202 ,   3.2376628 ,  -2.0905313 ,  -3.7804592 ,
         -5.883827  ,   0.08440585],
       [ -8.921265  ,  -3.7802005 , -10.13493   ,   3.596607  ,
          2.149397  ,   0.65961874],
       [  7.0937    ,   1.1944131 ,  -4.971827  ,  -7.036687  ,
         -9.642519  , -12.8935    ],
       [ -7.4679418 ,  -1.9035491 , -11.505154  ,   1.5854244 ,
          4.2069497 ,  -1.8249745 ],
       [ -8.43745   ,  -3.5732746 ,  -9.530142  ,   3.1464403 ,
          1.8868854 ,   0.9559525 ],
       [ -5.4032817 ,  -1.5574425 ,  -1.3243635 ,  -0.26287726,
         -2.2963195 ,   2.1839383 ],
       [ -8.841539  ,  -3.75239   , -10.036861  ,   3.537414  ,
          2.1129875 ,   0.69975674],
       [  7.1128926 ,   1.2228429 ,  -4.9246383 ,  -7.1836047 ,
         -9.738779  , -12.832937  ],
       [  7.1083765 ,   1.2992601 ,  -4.887204  ,  -7.4400754 ,
         -9.9240265 , -12.74967   ],
       [ -3.757662  ,  -0.19508672,   5.1231823 ,  -3.546014  ,
         -9.0939865 ,  -1.0

In [19]:
import numpy as np

y_pred = model.predict(val_ds)

# Ubah output prediksi menjadi label kelas
label_kelas = ['Sawah', 'Perumahan', 'Taman', 'Ruko', 'Kantor', 'Pasar']
y_pred_label = [label_kelas[np.argmax(prediksi)] for prediksi in y_pred]

# Tampilkan hasil prediksi
print(y_pred_label)


['Pasar', 'Pasar', 'Taman', 'Perumahan', 'Ruko', 'Sawah', 'Sawah', 'Perumahan', 'Sawah', 'Ruko', 'Kantor', 'Ruko', 'Ruko']


In [20]:
for x in y_pred_label:
    print(x)

Pasar
Pasar
Taman
Perumahan
Ruko
Sawah
Sawah
Perumahan
Sawah
Ruko
Kantor
Ruko
Ruko


In [21]:
 [ -3.4406643 ,  -0.51060563,  -4.1595945 ,   2.022017  ,
         -0.9544041 ,  -2.0240772 ]]

SyntaxError: unmatched ']' (142618287.py, line 2)

In [ ]:
['Sawah', 'Perumahan', 'Taman', 'Ruko', 'Kantor', 'Pasar']